In [1]:
!pip install langchain langchain-community langchainhub qdrant-client langchain-openai bs4 -qq

  Using cached langchainhub-0.1.15-py3-none-any.whl.metadata (621 bytes)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached types_requests-2.31.0.20240406-py3-none-any.whl.metadata (1.8 kB)
Using cached langchainhub-0.1.15-py3-none-any.whl (4.6 kB)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Using cached types_requests-2.31.0.20240406-py3-none-any.whl (15 kB)


Based on [Langchain Quickstart](https://python.langchain.com/docs/use_cases/question_answering/quickstart/)

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
import getpass
import os

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo")

In [7]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [14]:
docs[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

In [17]:
chars = len(docs[0].page_content)
approx_tokens = chars*3/4
approx_tokens

32348.25

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [56]:
splits[:5]

[Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n

In [33]:
vectorstore = Qdrant.from_documents(documents=splits, embedding=OpenAIEmbeddings(), output_parser=StrOutputParser(), collection_name="lilianweng", location=":memory:")

2024-04-12 06:55:32.302 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


In [35]:
retriever = vectorstore.as_retriever()

In [37]:
prompt = hub.pull("rlm/rag-prompt")

In [47]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [48]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
rag_chain.invoke("What do we about planning and subgoals?")

'In planning and subgoals, tasks are decomposed into manageable steps or subgoals to simplify complex problems and enhance task execution. Techniques such as Chain of Thought (CoT) and Tree of Thoughts (ToT) enable an agent to think step-by-step through a problem, exploring multiple reasoning paths. Additionally, external tools like classical planners can be used for long-horizon planning, interfacing through Planning Domain Definition Language (PDDL) to translate and execute plans.'